<a href="https://colab.research.google.com/github/anjunjin/PFE-ING3-IA/blob/master/LinkWithTwitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation de la library *python-dotenv*

In [1]:
!pip install python-dotenv

Importation des libraries

In [2]:
import tweepy
import pickle
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from dotenv import dotenv_values
import requests
from IPython.display import JSON

Liaison au Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = "/content/drive/My Drive/PFE/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Récupération dans ***data*** des donnés nécessaires de tableauSites.csv

In [4]:
data = pd.read_csv(DATA_PATH+"tableauSites.csv", encoding='utf-8').drop(['Unnamed: 0', 'id'], axis=1).values.tolist()

Récupération des clés pour se connecter au compte Twitter Developers et utiliser son API

In [5]:
API_TOKEN = pd.read_csv(DATA_PATH+"Clés.csv", encoding='utf-8', sep=',')['Bearer Token'][0]
API_KEY = pd.read_csv(DATA_PATH+"Clés.csv", encoding='utf-8', sep=',')['API Key'][0]
API_SECRET = pd.read_csv(DATA_PATH+"Clés.csv", encoding='utf-8', sep=',')['API Key Secret'][0]

Authentifiation au compte Twitter Developers et récupération de l'API tweepy

In [6]:
auth = tweepy.AppAuthHandler(API_KEY, API_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

Création d'une classe TwitterUser contenant les fonctions de récupération de données (utilisateurs, tweets, ...)

In [7]:
    class TwitterUser():
        _screen_name = ""
        _id = ""
        _user_data = None
        followers = []
        following = []
        tweet = []
        like = []
        request_count = 0

        def __init__(self, sname = "", _json = {}):
            if (sname != ""):
                self._screen_name = sname
                self.get_info()
            if (_json != {}):
                self._json = _json

        def get_info(self):
            try:
                self._user_data = api.get_user(screen_name=self._screen_name)
            except Exception as e:
                print("Exception get_info")

        def get_all_data(self):
            self.get_tweets()
            # self.get_followers()
            # self.get_following()
            # self.get_like()

        def set_name(self, name):
            self._screen_name = name

        def set_id(self, _id):
            self._id = _id

        def get_followers(self):
            with tqdm(total=self._user_data.followers_count, desc="get_followers for " + self._user_data.screen_name) as pbar:
                for page in tweepy.Cursor(api.followers, screen_name=self._screen_name, count=200).pages():
                    self.followers.extend(page)
                    pbar.update(len(page))


        def get_following(self):
            with tqdm(total=self._user_data.friends_count, desc="get_following for " + self._user_data.screen_name) as pbar:
                for page in tweepy.Cursor(api.friends, screen_name=self._screen_name, count=200).pages():
                    self.following.extend(page)
                    pbar.update(len(page))

        # on ne peut prendre que les 3000 derniers
        def get_tweets(self):
            try:
                with tqdm(total=1000, desc="get_tweets for " + self._user_data.screen_name) as pbar:
                    for page in tweepy.Cursor(api.user_timeline, screen_name=self._screen_name, count=200).pages():
                        self.tweet.extend(page)
                        pbar.update(len(page))
            except Exception as e:
                print("Exception get_tweets")

        # on ne peut prendre que les 3000 derniers
        def get_like(self):
            if self._user_data.protected:
                print("can get like from " + self._user_data.screen_name + ", the account is protected")
            else:
                with tqdm(total=3000, desc="get_likes for " + self._user_data.screen_name) as pbar:
                    for page in tweepy.Cursor(api.favorites, screen_name=self._screen_name, count=200).pages():
                        self.like.extend(page)
                        pbar.update(len(page))


Division de la liste des site en plusieurs sous liste pour ne pas exéder la RAM alloué par Google Colab 

In [8]:
def split_list(a_list):
    half = len(a_list)//2
    return a_list[:half], a_list[half:]

dataA, dataB = split_list(data)
data1, data2 = split_list(dataA)
data3, data4 = split_list(dataB)

Ajout de tous les tweets dans un tableau qui va contenir les données importantes de chaque tweet.

In [9]:
list_tweet = []

for site in data1:
  user = TwitterUser(sname= site[2])
  user.get_all_data()
  for tweet in user.tweet:
    list_tweet.append([tweet.user.id, tweet.id, tweet.text, tweet.retweet_count, tweet.favorite_count, site[1]])

tableauTweets = pd.DataFrame(list_tweet, columns=['user_id', 'id_tweet', 'tweet', 'retweet_count', 'favorite_count', 'label_liability'])
tableauTweets

get_tweets for france_soir: 3213it [00:07, 408.84it/s]
get_tweets for ThierryRegenere:  38%|███▊      | 375/1000 [00:00<00:01, 464.70it/s]


Exception get_info
Exception get_tweets


get_tweets for EmaKrusi:  32%|███▏      | 315/1000 [00:01<00:02, 303.49it/s]
get_tweets for LLP_Le_Vrai: 3040it [00:10, 276.84it/s]
get_tweets for covid_infos:  27%|██▋       | 271/1000 [00:00<00:02, 312.69it/s]
get_tweets for cab2626: 1911it [00:04, 392.57it/s]
get_tweets for lamentableFR: 2123it [00:04, 508.14it/s]
get_tweets for Elishean: 3250it [00:06, 477.09it/s]
get_tweets for DrMo7oG: 2984it [00:08, 372.53it/s]
get_tweets for yetiblog: 3114it [00:09, 341.80it/s]
get_tweets for lepolitique_fr:   0%|          | 1/1000 [00:00<05:35,  2.98it/s]
get_tweets for Les_Repliques: 3250it [00:07, 410.08it/s]


Exception get_info
Exception get_tweets


get_tweets for Jnewsfoot: 3192it [00:07, 442.52it/s]
get_tweets for JournalElysee: 1107it [00:02, 410.21it/s]


...

Exception get_info
Exception get_tweets


get_tweets for FoxNews: 3250it [00:06, 466.96it/s]
get_tweets for i24NEWS_FR: 3250it [00:12, 259.07it/s]
get_tweets for Lesjoursfr: 3250it [00:09, 339.73it/s]
get_tweets for nextinpact: 3250it [00:07, 409.35it/s]
get_tweets for Rue89Bordeaux: 3249it [00:07, 446.51it/s]
get_tweets for lesoir: 3250it [00:07, 438.96it/s]
get_tweets for conspiration: 3248it [00:09, 359.63it/s]
get_tweets for Korben: 2017it [00:04, 441.56it/s]
get_tweets for leLab_E1: 3200it [00:06, 468.94it/s]
get_tweets for Qofficiel: 3250it [00:07, 432.13it/s]


Exception get_info
Exception get_tweets


get_tweets for lundimat1: 3228it [00:08, 378.03it/s]
get_tweets for Fakir_: 3248it [00:09, 339.05it/s]
get_tweets for PaulJorion: 3249it [00:07, 449.77it/s]
get_tweets for cahiersdufoot: 3240it [00:08, 377.68it/s]


,user_id,id_tweet,tweet,retweet_count,favorite_count,label_liability
0,78957336,1464256415372034074,👀 Des professeurs niçois ont découvert des cam...,38,72,2
1,78957336,1464229853205602312,RT @CStrateges: Revue de presse :\nL'épidémiol...,235,0,2
2,78957336,1464228383085240320,🏎️ Objectif zéro mort sur les routes en 2050 :...,60,89,2
3,78957336,1464221914847657988,"Et notre entretien, à voir ou revoir 👇\nhttps:...",29,73,2
4,78957336,1464221524102160427,"📣 ""Nous voici donc entrés dans une nouvelle zo...",68,133,2
...,...,...,...,...,...,...
10378692,96419861,1217018179823730689,"Néanmoins, dans le Top 20, quelques clubs font...",2,5,4
10378693,96419861,1217017969366249473,"Selon le rapport annuel Deloitte, les revenus ...",16,38,4
10378694,96419861,1216978446565494784,"RT @FlorentToniutti: Coucou, c'est bien @floto...",468,0,4
10378695,96419861,1216814345390383104,@yvanmaugere Nous n'avons reçu qu'un message :...,0,0,4


In [ ]:
list_tweet = []

for site in data2:
  user = TwitterUser(sname= site[2])
  user.get_all_data()
  for tweet in user.tweet:
    list_tweet.append([tweet.user.id, tweet.id, tweet.text, tweet.retweet_count, tweet.favorite_count, site[1]])

tableauTweets = pd.DataFrame(list_tweet, columns=['user_id', 'id_tweet', 'tweet', 'retweet_count', 'favorite_count', 'label_liability'])
tableauTweets

get_tweets for acrimed_info: 3247it [00:09, 329.80it/s]
get_tweets for Charlie_Hebdo_: 3250it [00:06, 472.78it/s]
get_tweets for EsanteFR: 3200it [00:06, 470.65it/s]
get_tweets for PasseportSante: 3250it [00:07, 422.60it/s]
get_tweets for Morandinisante: 3171it [00:07, 428.46it/s]
get_tweets for lesucretvous: 3198it [00:09, 328.63it/s]
get_tweets for electroallergik: 3141it [00:08, 358.61it/s]
get_tweets for Allodocteurs: 3248it [00:07, 422.56it/s]
get_tweets for AlterEco_: 3244it [00:09, 344.15it/s]
get_tweets for mesopinions_com: 3246it [00:07, 442.68it/s]
get_tweets for Change: 3250it [00:07, 440.36it/s]


Exception get_info
Exception get_tweets


get_tweets for Canardpcredac: 3249it [00:07, 408.93it/s]
get_tweets for Confidentielles: 3200it [00:06, 498.23it/s]
get_tweets for dirtybiology: 3224it [00:07, 419.33it/s]
get_tweets for epenser: 3250it [00:07, 445.70it/s]
get_tweets for Clubic: 3250it [00:06, 465.77it/s]
get_tweets for journalDfemmes: 3234it [00:11, 289.67it/s]
get_tweets for commentcamarche: 3250it [00:07, 442.21it/s]
get_tweets for JVCom: 3250it [00:07, 425.50it/s]
get_tweets for astucenaturelle:  31%|███       | 310/1000 [00:00<00:01, 366.90it/s]
get_tweets for Santeplusmag: 3200it [00:06, 468.79it/s]


Exception get_info
Exception get_tweets


get_tweets for humourdedroite: 3235it [00:10, 306.47it/s]
get_tweets for Maitre_Eolas: 3249it [00:07, 434.27it/s]


Exception get_info
Exception get_tweets


get_tweets for VosgesMatin: 3250it [00:12, 253.72it/s]
get_tweets for VICEfr: 3249it [00:06, 471.61it/s]
get_tweets for VICE: 3250it [00:09, 352.62it/s]
get_tweets for USATODAY: 3250it [00:07, 415.95it/s]
get_tweets for TVMAG: 3250it [00:07, 412.99it/s]
get_tweets for washingtonpost: 3250it [00:07, 443.46it/s]
get_tweets for WSJ: 3250it [00:06, 474.01it/s]
get_tweets for nytimes: 3250it [00:08, 380.35it/s]
get_tweets for theintercept: 3250it [00:06, 468.29it/s]
get_tweets for Independent: 3250it [00:07, 428.04it/s]
get_tweets for heraldscotland: 3250it [00:07, 407.92it/s]
get_tweets for guardian: 3250it [00:07, 448.51it/s]
get_tweets for TheEconomist: 3250it [00:07, 459.84it/s]
get_tweets for dailytelegraph: 3250it [00:06, 468.87it/s]
get_tweets for TF1: 3246it [00:08, 396.49it/s]
get_tweets for TETUmag: 3246it [00:07, 438.35it/s]
get_tweets for Telerama: 3250it [00:08, 371.43it/s]
get_tweets for TeleLoisirs: 3250it [00:07, 463.03it/s]
get_tweets for Tele7: 3250it [00:08, 370.09it/s]
g

Réinitialisation du fichier PauchBas.pickle

In [ ]:
import pickle

with open(DATA_PATH+"PaucBas.pickle", "wb") as obj_file:
    pickle.dump([], obj_file, -1)

Sauvegarde des données reçu pour chaque utilisateur Twitter recherché

In [ ]:
# with open(DATA_PATH+"PaucBas.pickle", "ab") as obj_file:
#   for site in data1:
#     user = TwitterUser(sname= site[2])
#     user.get_all_data()
#     pickle.dump([user], obj_file, pickle.HIGHEST_PROTOCOL)

In [ ]:
# with open(DATA_PATH+"PaucBas.pickle", "ab") as obj_file:
#   for site in data2:
#     user = TwitterUser(sname= site[2])
#     user.get_all_data()
#     pickle.dump([user], obj_file, pickle.HIGHEST_PROTOCOL)